# **Data Preprocessing**

### **Analize Data**

Pada proses ini kita akan menyiapkan data dan library yang akan digunakan dan juga akan menganalisa data apa saja yang akan digunakan

In [138]:
# importing library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from wordcloud.wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [139]:
# membaca dataset
df = pd.read_csv("chatgpt_reviews.csv")

# menampilkan data
df.head()

,date,title,review,rating
0,2023-05-21 16:42:24,Much more accessible for blind users than the ...,Up to this point I’ve mostly been using ChatGP...,4
1,2023-07-11 12:24:19,"Much anticipated, wasn’t let down.",I’ve been a user since it’s initial roll out a...,4
2,2023-05-19 10:16:22,"Almost 5 stars, but… no search function",This app would almost be perfect if it wasn’t ...,4
3,2023-05-27 21:57:27,"4.5 stars, here’s why","I recently downloaded the app and overall, it'...",4
4,2023-06-09 07:49:36,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4


In [140]:
# melihat banyak data setiap class
df.rating.value_counts()

rating
5    1134
1     495
4     304
3     220
2     139
Name: count, dtype: int64

### **Feature Selection**

Pada tahapan ini kita akan melakukan seleksi lagi untuk fitur fitur yang akan digunakan saja. Untuk kasus text mining kita hanya menggunakan fitur reviews sebagai data x dan rating sebagai y. 

In [141]:
# menghilangkan fitur yang tidak diperlukan
df.drop(columns=["title", "date"], inplace=True)

df.head()

,review,rating
0,Up to this point I’ve mostly been using ChatGP...,4
1,I’ve been a user since it’s initial roll out a...,4
2,This app would almost be perfect if it wasn’t ...,4
3,"I recently downloaded the app and overall, it'...",4
4,I appreciate the devs implementing Siri suppor...,4


### **Duplicated Data**

Pada tahapan ini kita akan menganalisa apakah ada data yang terduplikasi atau tidak. Jika iya maka kita perlu menghapus semuanya yang terduplikasi dan menyisakan satu. Tujuan dari hal ini agar model itu tidak belajar pada data yang sama. Sehingga jikalau dia belajar data yang sama akan menimbulkan model itu menghafal bukan mencari pola. Jadi akan menimbulkan overfitting.

In [142]:
# untuk melihat data yang terduplikasi
df[df.duplicated()]

,review,rating
1351,Thanks,5
1508,Please,5
1783,Love it,5
1800,First,5
1801,First,5
1802,First,5
1827,Why?,1
1853,Very good,5
1866,Pretty good,5
1869,Great,5


berdasarkan data yang diatas, kita dapat melihat bahwa ada data yang terduplikasi. Maka tahapan selanjutnya adalah kita menghilangkan data yang terduplikasi tersebut.

In [143]:
# menghilangkan data yang terduplikasi
df.drop_duplicates(inplace=True)

df[df.duplicated()]

,review,rating


### **Missing Value**

Pada tahapan ini kita akan mencari apakah ada data yang hilang atau tidak. Data yang hilang akan sangat memengaruhi proses  training. Jika pada text data, maka model akan mengira "Nan" itu sebagai string. Hal ini akan sangat memengaruhi model nya nanti. Jika pada label nya maka yang terjadi, model akan mengira "Nan" itu sebagai label baru nya.


In [144]:
# banyaknya data kosong jenis 'nan'
df.isna().sum()

review    0
rating    0
dtype: int64

In [145]:
# banyaknya data kosong jenis 'null'
df.isnull().sum()

review    0
rating    0
dtype: int64

In [146]:
# melihat nilai unik pada label
df.rating.unique()

array([4, 1, 3, 5, 2], dtype=int64)

### **Case Folding**

In [147]:
# sebelum di case folding
df.review.loc[649]

'This app really helped! But I noticed one imperfection, there is no bookmarks, when I want to save one GPT response I need copy and transfer to notes! I would like to save GPT response to bookmarks!! Thanks'

Pada tahapan ini kita akan menghilangkan karakter yang tidak diperlukan seperti emoji, hashtag, tag, dan lain lain. Hal ini dikarenakan model kita tidak akan mengetahui pada data yang spesifik. Karena data yang terlalu spesifik tidak akan membuat pola apapun

In [148]:
def case_folding(text):
    # menhilangkan karakter tag
    text = re.sub(r"@[A-Za-z0-9_]+", "", text)

    # menhilangkan karakter hashtag
    text = re.sub(r"#\w+", "", text)

    # menhilangkan karakter enter
    text = re.sub(r"\n+", "", text)

    # menhilangkan karakter angka
    text = re.sub(r"\d+", "", text)

    # menhilangkan karakter angka
    text = re.sub(r"RT[\s]+", "", text)

    # menhilangkan link
    text = re.sub(r"https?://\S+", "", text)

    # menghilangkan simbol
    text = text.translate(str.maketrans("", "", string.punctuation))

    # menghilangkan emoji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" # emoji
                               u"\U0001F300-\U0001F5FF" # simbol
                               u"\U0001F680-\U0001F6FF" # emoji ios
                               u"\U00010000-\U0010ffff" # emoji
                               "]+", flags=re.UNICODE)
    
    text = emoji_pattern.sub("", text)

    return text.lower()

# melakukan case folding
df.review = df.review.apply(case_folding)

df.review

0       up to this point i’ve mostly been using chatgp...
1       i’ve been a user since it’s initial roll out a...
2       this app would almost be perfect if it wasn’t ...
3       i recently downloaded the app and overall its ...
4       i appreciate the devs implementing siri suppor...
                              ...                        
2287    this has to be a beginning to something crazy ...
2288    i’ve been using chat and have been a proud pre...
2289    the chatgpt ios app is an outstanding product ...
2290    sam altman’s blatant attempt at regulatory cap...
2291                                                     
Name: review, Length: 2262, dtype: object

In [149]:
# setelah di case folding
df.review.loc[649]

'this app really helped but i noticed one imperfection there is no bookmarks when i want to save one gpt response i need copy and transfer to notes i would like to save gpt response to bookmarks thanks'

### **Tokenize**

Pada tahapan ini kita akan memisahkan dari kalimat menjadi kata kata. Tujuannya agar dapat diperhitungkan untuk pemobobotan kata

In [150]:
# fungsi yang akan melakukan tokenize
def tokenize(text):
    return word_tokenize(text)

# mengimplementasikan setiap item
df.review = df.review.apply(tokenize)

df.review

0       [up, to, this, point, i, ’, ve, mostly, been, ...
1       [i, ’, ve, been, a, user, since, it, ’, s, ini...
2       [this, app, would, almost, be, perfect, if, it...
3       [i, recently, downloaded, the, app, and, overa...
4       [i, appreciate, the, devs, implementing, siri,...
                              ...                        
2287    [this, has, to, be, a, beginning, to, somethin...
2288    [i, ’, ve, been, using, chat, and, have, been,...
2289    [the, chatgpt, ios, app, is, an, outstanding, ...
2290    [sam, altman, ’, s, blatant, attempt, at, regu...
2291                                                   []
Name: review, Length: 2262, dtype: object

### **Normalize**

Pada tahapan ini kita akan melakukan normalisasi untuk kata kata yang bisa dibilang gaul atau kata kata yang kurang tepat menjadi kata kata yang umum dan tepat.

In [151]:
# berisi kata yang perlu dinormalisasi
norm_word = {"nt":"not", "t":"not", "iam":"i"}

# melakukan normalisasi
def normalize(doc):
    return [norm_word[word] if word in norm_word else word for word in doc]

# mengimplementasikan pada setiap doc
df.review = df.review.apply(normalize)

df.review

0       [up, to, this, point, i, ’, ve, mostly, been, ...
1       [i, ’, ve, been, a, user, since, it, ’, s, ini...
2       [this, app, would, almost, be, perfect, if, it...
3       [i, recently, downloaded, the, app, and, overa...
4       [i, appreciate, the, devs, implementing, siri,...
                              ...                        
2287    [this, has, to, be, a, beginning, to, somethin...
2288    [i, ’, ve, been, using, chat, and, have, been,...
2289    [the, chatgpt, ios, app, is, an, outstanding, ...
2290    [sam, altman, ’, s, blatant, attempt, at, regu...
2291                                                   []
Name: review, Length: 2262, dtype: object

### **Stopwords**

Pada tahapan ini kita akan menghilangkan kata yang memiliki makna yang dependen. Setiap kata itu harus memiliki makn a secara independen dengan tujuan agar data kita benar benar memiliki pola yang jelas. Jikalau depeden maka hasil dari data tersebut pastinya tidak memiliki pola sehingga akan buruk untuk dilakukan training pada model. Contoh kata yang harus dihilangkan adalah yang, dan, itu, dll.

In [152]:
# menyimpan kata yang harus dihilangkan
stopw = set(stopwords.words())

# menyeleksi kata 
def stopword(doc):
    return [word for word in doc if word not in stopw]

# mengimplementasikan untuk setiap doc
df.review = df.review.apply(stopword)

df.review

0       [point, ’, chatgpt, windows, desktop, google, ...
1       [’, user, ’, initial, roll, waiting, mobile, a...
2       [app, perfect, ’, “, search, ”, function, imag...
3       [recently, downloaded, app, great, platform, e...
4       [appreciate, devs, implementing, siri, support...
                              ...                        
2287    [beginning, crazy, letting, ’, ’, day, ima, fi...
2288    [’, chat, proud, premium, subscriber, awhile, ...
2289    [chatgpt, ios, app, outstanding, product, seam...
2290    [sam, altman, ’, blatant, attempt, regulatory,...
2291                                                   []
Name: review, Length: 2262, dtype: object

### **Stemming**

Pada proses ini kita akan melakukan pengubahan kata yang tidak asli atau imbuhan menjadi kata aslinya. Misal pada data bahasa indonesia, kata imbuhan akan dijadikan kata yang tidak memiliki imbuhan sama sekali. Kalau dalam bahasa inggris, dari kata verb 3 atau 2 menjadi verb 1. Tujuannya adalah agar pada kata yang memiliki makna yang sama ini, meggabung jadi satu. Jika tidak diubah, model akan mengira kata dengan verb 1 dan kata dengan verb 2 pada kata yang sama, model mengira memiliki makna yang berbeda

In [153]:
# object yang akan melakukan stemming
stemmer = PorterStemmer()

# fungsi yang akan mengembalikan document yang sudah di stemming
def stemming(doc):
    return [stemmer.stem(word) for word in doc]

# mengaplikasikannya pada setiap document
df.review = df.review.apply(stemming)

df.review

0       [point, ’, chatgpt, window, desktop, googl, ch...
1       [’, user, ’, initi, roll, wait, mobil, applic,...
2       [app, perfect, ’, “, search, ”, function, imag...
3       [recent, download, app, great, platform, excel...
4       [appreci, dev, implement, siri, support—it, en...
                              ...                        
2287    [begin, crazi, let, ’, ’, day, ima, find, comm...
2288    [’, chat, proud, premium, subscrib, awhil, hel...
2289    [chatgpt, io, app, outstand, product, seamless...
2290    [sam, altman, ’, blatant, attempt, regulatori,...
2291                                                   []
Name: review, Length: 2262, dtype: object

pada tahapan akhir ini, saya perlu untuk mengambil dua class saja dan menyeimbangkan banyaknya data

In [155]:
# melakukan sampling dan penyeimbangan data
df = pd.concat([df[df.rating==5][:500], df[df.rating==1]]).sample(frac=1, random_state=123)

df.head()

,review,rating
282,"[’, openai, team, beauti, app, make, thing, ph...",5
709,"[surpris, appl, approv, process, violat, polic...",1
2272,"[bewar, app, requir, person, info, access, sen...",1
2251,"[send, code, verifi, number]",1
293,"[ui, clean, haptic, feedback, appreci, hope, w...",5
